In [3]:
print("hello")

hello


In [5]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "D:\Generative.AI\Medical_Chatboat_Generative-AI\Data\Medical_book.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()


In [ ]:
#docs

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)

texts_chunk = text_splitter.split_documents(docs)

In [7]:
len(texts_chunk)

37348

In [8]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

In [9]:
result = embeddings.embed_query("Hello, world!")
print(len(result))

1536


In [10]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec



import os
from dotenv import load_dotenv

load_dotenv()

pinecone_api_key = os.environ.get("PINECONE_API_KEY")



# Step 2: Initialize the Pinecone client
pc = Pinecone(api_key=pinecone_api_key)

# Step 3: Define index name
index_name = "medical-chatbot"

# Step 4: Create the index
pc.create_index(
    name=index_name,
    dimension=1536,  # --> set according to your embedding vector size
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)


c:\Users\ASUS\anaconda3\envs\medical_chat\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


## Our Vector Embeddings Store in PineconeDB

* It means jitne aapke text_chunk hai utne hi  Pinecone vector DB mai store honge like 42207

In [11]:
from langchain_pinecone import PineconeVectorStore

doc_search = PineconeVectorStore.from_documents(
    documents=texts_chunk,
    index_name = index_name,
    embedding = embeddings
)

## Load the existing index

In [12]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name= index_name,
    embedding=embeddings
)

In [10]:
docsearch

## Retriver-->

In [13]:
retriever = docsearch.as_retriever(search_type = "similarity", search_kwargs = {"k" : 3})  ## it give you 3 relevant answer

In [14]:
result = retriever.invoke("what is Acne")

In [15]:
result

[Document(metadata={'page': 241.0, 'source': 'D:\\Generative.AI\\Medical_Chatboat_Generative-AI\\Data\\Medical_book.pdf'}, page_content='Acne —A skin condition in which raised bumps,'),
 Document(metadata={'page': 37.0, 'source': 'D:\\Generative.AI\\Medical_Chatboat_Generative-AI\\Data\\Medical_book.pdf'}, page_content='Description\nAcne vulgaris, the medical term for common acne, is'),
 Document(metadata={'page': 38.0, 'source': 'D:\\Generative.AI\\Medical_Chatboat_Generative-AI\\Data\\Medical_book.pdf'}, page_content='of diag-nosing and treating acne.')]

##  I want to relevant/crucial output. So , now we connect my LLms model 

In [16]:
import os
from dotenv import load_dotenv

load_dotenv()

openai_api_key = os.environ.get("OPENAI_API_KEY")

In [17]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model = "gpt-4o-2024-08-06")

In [18]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate



system_prompt = (
    " You are an assistant for question-answer tasks"
    "Use the following pieces of retrieved context to answer"
    "the question. If you don't know the answer , say that you "
    " don't know. Use three sentence maximum and keep the "
    "answer concise. \n\n"
    "{context}"  
)


prompt_template = ChatPromptTemplate.from_messages(
    [

    ("system", "system_prompt"),
    ("human", "{context}")
]

)



## Chain

In [19]:
question_answer_chain = create_stuff_documents_chain(llm,prompt_template)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [20]:
response = rag_chain.invoke({"input" : "What is Acne .?"})
print(response["answer"])

Acne vulgaris, commonly referred to as acne, is a prevalent skin condition that occurs when hair follicles become clogged with oil and dead skin cells. It often manifests as raised bumps on the skin, such as whiteheads, blackheads, pimples, and sometimes cysts or nodules. Acne typically appears on the face, forehead, chest, upper back, and shoulders.

The condition is most common among teenagers, though it can affect individuals of all ages. Hormonal changes, such as those occurring during puberty, menstruation, or pregnancy, often exacerbate acne. Other contributing factors include genetics, certain medications, diet, and stress.

Diagnosing acne involves a physical examination by a healthcare professional, who will assess the severity and type of acne lesions present. The diagnosis may include determining the underlying causes, such as hormonal imbalances or lifestyle factors.

Treatment for acne varies depending on its severity and can include:

1. **Topical Treatments:** Over-the-c

In [21]:
response = rag_chain.invoke({"input" : "What is the harmfull affect of paractamol .?"})
print(response["answer"])

When using any medication, it's crucial to follow the prescribed dosage and guidelines provided by your healthcare provider. Overuse or misuse of medication can lead to serious side effects, including potentially life-threatening conditions. Here are a few general points to consider:

1. **Adverse Reactions**: While harmful effects are uncommon, they can occur. It’s important to be aware of any unusual symptoms or allergic reactions and to seek medical attention if they arise.

2. **Interactions**: Some medications can interact with other drugs, supplements, or certain foods, which can increase the risk of side effects. Always inform your healthcare provider about any other substances you are taking.

3. **Individual Response**: People may respond differently to medications based on factors like age, weight, genetics, and overall health. Monitoring how the medication affects you is essential.

4. **Avoid Certain Activities**: Until you know how the medication affects you, avoid activit